In [1]:
import openpyxl
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import folium
from folium import plugins
from folium.plugins import HeatMap, MarkerCluster
from IPython.display import IFrame
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
from math import radians, sin, cos, sqrt, atan2
import branca
import requests

In [2]:
def get_elevation(lat, lon):
    url = f"https://api.open-elevation.com/api/v1/lookup?locations={lat},{lon}"
    response = requests.get(url).json()
    return response['results'][0]['elevation'] if "results" in response else None

def haversine_distance(point1: Point, point2: Point) -> float:
    R = 6371.0
    lon1, lat1 = point1.x, point1.y
    lon2, lat2 = point2.x, point2.y
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1
    a = sin(delta_lat / 2)**2 + cos(lat1) * cos(lat2) * sin(delta_lon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def create_point(row):
    return Point(row['longitude'], row['latitude'])

In [3]:
# Load the Excel files into Pandas DataFrames
gis_weather_station = pd.read_csv('./data/gis_weatherstation_shape_2024_10_04.csv')
station_summary_snapshot = pd.read_csv('./data/src_wings_meteorology_station_summary_snapshot_2023_08_02.csv')
windspeed_snapshot = pd.read_csv('./data/src_wings_meteorology_windspeed_snapshot_2023_08_02.csv')

src_vri_snapshot = pd.read_csv('./data/src_vri_snapshot_2024_03_20.csv')
nam = pd.read_csv('./data/nam.csv')
san_diego_county_gpd = gpd.read_file("data/San_Diego_County_Boundary.geojson")

In [4]:
gis_weather_station['geometry'] = gis_weather_station['shape'].apply(wkt.loads)
gis_weather_station_gpd = gpd.GeoDataFrame(gis_weather_station, geometry='geometry', crs=f"EPSG:{gis_weather_station['shape_srid'][0]}")

src_vri_snapshot['geometry'] = src_vri_snapshot['shape'].apply(wkt.loads)
src_vri_snapshot_gpd = gpd.GeoDataFrame(src_vri_snapshot, geometry='geometry', crs=f"EPSG:{src_vri_snapshot['shape_srid'][0]}")

# Get ESPG:4326
nam_crs = src_vri_snapshot['shape_srid'][0]

# Create NAM Geodataframe
nam['geometry'] = nam.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
nam_gpd = gpd.GeoDataFrame(nam, geometry='geometry', crs=nam_crs)

print(f"Weather Station CRS:    {gis_weather_station_gpd.crs}")
print(f"VRI Polygon CRS:        {src_vri_snapshot_gpd.crs}")
print(f"NAM CRS:                {nam_gpd.crs}")
print(f"San Diego County:       {san_diego_county_gpd.crs}")

Weather Station CRS:    EPSG:4431
VRI Polygon CRS:        EPSG:4326
NAM CRS:                EPSG:4326
San Diego County:       EPSG:4326


In [5]:
gis_weather_station_gpd = gis_weather_station_gpd.to_crs(src_vri_snapshot_gpd.crs)

print(f"Weather Station CRS:    {gis_weather_station_gpd.crs}")
print(f"VRI Polygon CRS:        {src_vri_snapshot_gpd.crs}")
print(f"NAM CRS:                {nam_gpd.crs}")
print(f"San Diego County:       {san_diego_county_gpd.crs}")

Weather Station CRS:    EPSG:4326
VRI Polygon CRS:        EPSG:4326
NAM CRS:                EPSG:4326
San Diego County:       EPSG:4326


In [6]:
# Get elevation and nearest coastline for Weather Station
gis_weather_station_gpd['elevation_m'] = gis_weather_station_gpd.apply(lambda row: get_elevation(row['geometry'].y, row['geometry'].x), axis=1)

# Save file as CSV
gis_weather_station_gpd.to_csv("data/gis_weather_station_with_elevation.csv", index=False)

In [7]:
san_diego_county_gpd = san_diego_county_gpd[['geometry']]

In [8]:
# Filter NAM within the VRI polygons
nam_vri_gpd = gpd.sjoin(nam_gpd, src_vri_snapshot_gpd, how='inner', predicate='within').drop(columns=['index_right'])
nam_san_diego_county_gpd = gpd.sjoin(nam_gpd, san_diego_county_gpd, how='inner', predicate='within').drop(columns=['index_right'])
nam_vri_san_diego_county_gpd = gpd.sjoin(nam_san_diego_county_gpd, src_vri_snapshot_gpd, how='inner', predicate='within')

In [9]:
print(f"Number of Unique Points within VRI Polygon: {nam_vri_gpd[['geometry']].nunique()}")
print(f"Number of Unique Points within San Diego County: {nam_san_diego_county_gpd[['geometry']].nunique()}")
print(f"Number of Unique Points within VRI Polygon in San Diego County: {nam_vri_san_diego_county_gpd[['geometry']].nunique()}")
print(f"Number of Unique Points within VRI Polygon and Outside San Diego County: {nam_vri_gpd[['geometry']].nunique() - nam_vri_san_diego_county_gpd[['geometry']].nunique()}")

print(f"Shape of NAM data within VRI polgyon: {nam_vri_gpd.shape}")
print(f"Shape of NAM data within San Diego county: {nam_san_diego_county_gpd.shape}")

Number of Unique Points within VRI Polygon: geometry    994
dtype: int64
Number of Unique Points within San Diego County: geometry    3457
dtype: int64
Number of Unique Points within VRI Polygon in San Diego County: geometry    938
dtype: int64
Number of Unique Points within VRI Polygon and Outside San Diego County: geometry    56
dtype: int64
Shape of NAM data within VRI polgyon: (183622, 32)
Shape of NAM data within San Diego county: (576975, 5)


In [10]:
nam_vri_gpd = nam_vri_gpd[['latitude', 'longitude', 'date', 'average_wind_speed', 'geometry']]

nam_vri_san_diego_county_gpd_combined = gpd.GeoDataFrame(pd.concat([nam_vri_gpd, nam_san_diego_county_gpd], ignore_index=True))
nam_vri_san_diego_county_gpd_combined = nam_vri_san_diego_county_gpd_combined.drop_duplicates()

print(f"Number of Unique Points within all VRI Polygon and San Diego County: {nam_vri_san_diego_county_gpd_combined[['geometry']].nunique()}")
print(f"Shape of NAM data within all VRI Polygon and San Diego County: {nam_vri_san_diego_county_gpd_combined.shape}")

Number of Unique Points within all VRI Polygon and San Diego County: geometry    3513
dtype: int64
Shape of NAM data within all VRI Polygon and San Diego County: (586998, 5)


In [11]:
# Get the unique geometries within the VRI
nam_vri_san_diego_county_gpd_combined_geometry = nam_vri_san_diego_county_gpd_combined[['geometry']].drop_duplicates().reset_index(drop=True)
nam_vri_san_diego_county_gpd_combined_geometry['nam_elevation_m'] = nam_vri_san_diego_county_gpd_combined_geometry.apply(lambda row: get_elevation(row['geometry'].y, row['geometry'].x), axis=1)
nam_vri_san_diego_county_gpd_combined_geometry.head()

,geometry,nam_elevation_m
0,POINT (-117.10672 32.54224),8.0
1,POINT (-117.09059 32.54212),94.0
2,POINT (-117.07446 32.54202),47.0
3,POINT (-116.89679 32.55416),156.0
4,POINT (-116.97732 32.56846),152.0


In [12]:
# Add elevation to NAM data
nam_vri_san_diego_county_gpd_combined = nam_vri_san_diego_county_gpd_combined.merge(nam_vri_san_diego_county_gpd_combined_geometry, on='geometry', how='inner')
nam_vri_san_diego_county_gpd_combined.head()

,latitude,longitude,date,average_wind_speed,geometry,nam_elevation_m
0,32.542244,-117.106720,2012-09-14,12.527321,POINT (-117.10672 32.54224),8.0
1,32.542120,-117.090590,2012-09-14,12.835195,POINT (-117.09059 32.54212),94.0
2,32.542015,-117.074460,2012-09-14,12.932552,POINT (-117.07446 32.54202),47.0
3,32.554165,-116.896790,2012-09-14,13.933927,POINT (-116.89679 32.55416),156.0
4,32.568460,-116.977325,2012-09-14,12.176566,POINT (-116.97732 32.56846),152.0
...,...,...,...,...,...,...
586993,33.496647,-117.439760,2022-12-25,9.638989,POINT (-117.43976 33.49665),513.0
586994,33.496570,-117.423450,2022-12-25,10.497464,POINT (-117.42345 33.49657),599.0
586995,33.496513,-117.407135,2022-12-25,10.306699,POINT (-117.40714 33.49651),732.0
586996,33.496426,-117.390810,2022-12-25,9.881019,POINT (-117.39081 33.49643),814.0


In [14]:
# # Save file as CSV
nam_vri_san_diego_county_gpd_combined.to_csv("data/nam_with_elevation.csv")